## Load data

In [1]:
import sys
from config import dataset_dir
sys.path.append('{}/lovit_textmining_dataset'.format(dataset_dir))

from navernews_10days import get_news_paths
from soynlp.utils import DoublespaceLineCorpus

corpus_path = get_news_paths(date='2016-10-20')
corpus = DoublespaceLineCorpus(corpus_path, iter_sent=True)

print('num sents = {}'.format(len(corpus)))

soynlp=0.0.49
Dataset version
[navermovie_comments.data] is latest (0.0.1)
[navermovie_comments.models] is latest (0.0.1)
[navernews_10days.data] is latest (0.0.1)
[navernews_10days.models] is latest (0.0.1)
num sents = 223357


## Train cohesion score

corpus의 모든 문장에 대하여 어절의 왼쪽의 subwords를 카운팅합니다. 이를 위해서 defaultdict를 이용합니다. 

In [2]:
from collections import defaultdict
import sys

L = defaultdict(int)
for sent in corpus:
    for eojeol in sent.split():
        for e in range(1, len(eojeol)+1):
            subword = eojeol[:e]
            L[subword] += 1
print('num subword = %d' % len(L))

num subword = 662782


Cohesion 은 길이가 2 이상인 subword 에 대하여 정의가 되기 때문에 길이가 1 인 단어에 대해서는 0 을 return 합니다. 

또한 word 가 L 에 없는 경우 (빈도수가 min_count 이하이거나 아예 코퍼스에 등장하지 않았던 경우)에는 0.0 을 return 하는 예외 처리를 합니다. 

cohesion 은 결국 (word의 빈도수 / 맨 왼쪽의 글자 빈도수) 의 1 / (n-1) 승 입니다. pow 함수는 지수승 계산을 할 수 있도록 해줍니다. 

In [3]:
def get_cohesion(word):

    # 글자가 아니거나 공백, 혹은 희귀한 단어인 경우
    if (not word) or ((word in L) == False): 
        return 0.0

    n = len(word)
    if n == 1:
        return 0.0

    word_freq = L.get(word, 0)
    base_freq = L.get(word[:1], 0)

    if base_freq == 0:
        return 0.0
    else:
        return pow((word_freq / base_freq), 1 / (n - 1))

print('청와대 : ', get_cohesion('청와대'))

청와대 :  0.4877686086797539


어절 `아이오아이는`의 왼쪽에서부터 subword 를 자른 뒤, cohesion 을 계산합니다.

In [4]:
word = '아이오아이는'
for e in range(2, 7):
    sub = word[:e]
    print('{}: {:5}'.format(sub, get_cohesion(sub)))

아이: 0.14807442926505623
아이오: 0.0962206918738307
아이오아: 0.20118122417377401
아이오아이: 0.30039353721795514
아이오아이는: 0.26079046776641046


이후에 subwords 에 대한 cohesion 계산을 반복적으로 할테니, 미리 길이가 2 이상이고 빈도수가 10 이상인 subwords 의 cohesion score 를 계산해 둡니다.

In [5]:
cohesion_score = {}
for word, count in L.items():
    if count < 10 or len(word) < 2:
        continue
    cohesion_score[word] = get_cohesion(word)

print('n computed = {}'.format(len(cohesion_score)))

n computed = 92254


예시 단어들의 cohesion 을 실제로 계산해 봅시다. L+[R] 의 경계가 되는 지점들에서 cohesion 값이 하락함을 볼 수 있습니다. 

```
청와 = 0.522
청와대 = 0.722
```
 
청와라는 글자가 등장하면 대부분 청와대가 등장했기 때문에 '청와대'의 cohesion 이 '청와'의 cohesion 보다 큽니다. 

In [6]:
for word in ['청와대는', '민정수석이', '트와이스가']:
    for e in range(2, len(word)+1):
        sub = word[:e]
        print('{}: {:.3}'.format(sub, cohesion_score.get(sub, 0.0)))
    print()

청와: 0.238
청와대: 0.488
청와대는: 0.249

민정: 0.123
민정수: 0.346
민정수석: 0.493
민정수석이: 0.456

트와: 0.0944
트와이: 0.307
트와이스: 0.455
트와이스가: 0.372



## Branching Entropy & Accessor Variety

Accessor Variety 는 `청와` 라는 subword 의 좌/우에 등장하는 글자의 고유 개수를 그 값으로 지닙니다.

Left-side Accessor Variety ($av_l$) 는 `청와` 왼쪽의 고유 글자 개수를, Right-side Accessor Variety ($av_r$) 는 `청와` 오른쪽의 고유 글자 개수를 값으로 이용합니다.

Branching Entropy 는 고유 글자 개수 대신, 고유 글자들이 나타날 확률의 entropy 를 ($be_l, be_r)$ 로 이용합니다.

이를 계산하려면 `청와` 라는 subword 의 오른쪽, 왼쪽에 등장한 글자의 종류를 카운팅해야 합니다. 이를 위하여 dict dict 형식으로 카운터를 만듭니다. 

```
L[청와대][에] = 5
L[청와대][는] = 25, 

...

R[서][에] = 5   # 청와대에서: '에서 <- 서'
R[서는][에] = 5 # 청와대에서는: '에서는 <- 서는'
```

`-2` 를 이용하면 str 의 뒤에서 2 글자를 slice 할 수 있습니다.

In [7]:
'abcde'[-2:]

'de'

In [8]:
from collections import defaultdict

L = defaultdict(lambda: defaultdict(lambda: 0)) # 어린 --> 어린이
R = defaultdict(lambda: defaultdict(lambda: 0)) # 어린이 <-- 린이

for sent in corpus:
    for eojeol in sent.split():
        n = len(eojeol)

        if n < 2:
            continue

        # 어린 -> 어린이
        for e in range(2, n + 1):
            subword_from = eojeol[:e-1]# '어린'
            subword_to = eojeol[:e]    #'어린이'
            L[subword_from][subword_to] += 1

        # 린이 -> 어린이
        for b in range(2, n + 1):
            subword_from = eojeol[-b+1:]
            subword_to = eojeol[-b:]
            R[subword_from][subword_to] += 1

print('num L = {}'.format(len(L)))
print('num R = {}'.format(len(R)))

num L = 341009
num R = 378508


word 가 R 에 존재하지 않는 경우에는 0 을 할당하고, word 가 R 에 존재한다면 R[word] 의 value 인 dict 의 사이즈를 av_l 로 이용합니다. 

```python
av_l = 0 if not word in R else len(R[word])
```

같은 방식으로 av_r 도 정의할 수 있습니다.

```python
av_r = 0 if not word in L else len(L[word])
```


In [9]:
def get_accessor_variety(word):
    av_l = 0 if not word in R else len(R[word])
    av_r = 0 if not word in L else len(L[word])
    return (av_l, av_r)

print('word\t(av_l, av_r)\n%s' % ('-'*20))

for subword in ['연합', '연합뉴', '연합뉴스', '국방', '국방부', '국방부는', '국방장', '국방장관', '트와이', '트와이스']:
    av_l, av_r = get_accessor_variety(subword)
    print('{}\t({}, {})'.format(subword, av_l, av_r))

word	(av_l, av_r)
--------------------
연합	(27, 27)
연합뉴	(0, 1)
연합뉴스	(2, 18)
국방	(1, 30)
국방부	(1, 8)
국방부는	(0, 0)
국방장	(0, 1)
국방장관	(2, 11)
트와이	(0, 1)
트와이스	(1, 10)


Branching Entropy 는 `len(dict)` 를 `entropy` 값으로 바꾸면 됩니다.

```python
be_l = 0 if not word in R else entropy(R[word])
be_r = 0 if not word in L else entropy(L[word])    
```

In [10]:
import numpy as np

def get_branching_entropy(word):

    def entropy(extensions):
        '''extensions: dict[str]: int'''
        sum_ = sum(extensions.values())
        if sum_ == 0:
            return 0

        entropy = 0
        for v in extensions.values():
            prob = v / sum_
            entropy += (prob * np.log(prob))
        return -1 * entropy

    be_l = 0.0 if not word in R else entropy(R[word])
    be_r = 0.0 if not word in L else entropy(L[word])    

    return be_l, be_r

엔트로피가 높은 subword 는 좌/우에 등장할 다른 단어들의 종류가 다양하기 때문에 단어일 가능성이 높습니다.

단어의 경계 부분에서 entropy가 증가합니다. 

```
국방	(-0.000, 1.485)
국방부	(-0.000, 1.599)
국방부는	(0.000, 0.000)
국방장	(0.000, -0.000)
국방장관	(0.279, 1.819)
```

In [11]:
print('word\t(be_l, be_r)\n%s' % ('-'*20))

for subword in ['연합', '연합뉴', '연합뉴스', '국방', '국방부', '국방부는', '국방장', '국방장관', '트와이', '트와이스', '트와이스는']:
    be_l, be_r = get_branching_entropy(subword)
    print('{}\t({:.4}, {:.4})'.format(subword, be_l, be_r))

word	(be_l, be_r)
--------------------
연합	(2.012, 0.2129)
연합뉴	(0.0, -0.0)
연합뉴스	(0.6931, 1.971)
국방	(-0.0, 1.485)
국방부	(-0.0, 1.599)
국방부는	(0.0, 0.0)
국방장	(0.0, -0.0)
국방장관	(0.2788, 1.819)
트와이	(0.0, -0.0)
트와이스	(-0.0, 1.283)
트와이스는	(0.0, 0.0)


그렇기 때문에 길이가 1 인 글자의 entropy 는 항상 높습니다. 그래서 길이가 1인 L들은 큰 의미를 지니지 못합니다. 

단, 조사/어미처럼 길이가 1 인 경우가 많고, 어절의 오른쪽에 등장한다면 left-side entropy 는 이를 확인하는 좋은 정보가 됩니다.

In [12]:
for subword in ['은', '는', '이', '가', '에게', '에서']:
    be_l, be_r = get_branching_entropy(subword)
    print('{}\t({:.4}, {:.4})'.format(subword, be_l, be_r))

은	(4.967, 1.92)
는	(3.93, 1.899)
이	(5.045, 4.236)
가	(4.483, 3.788)
에게	(3.622, 1.049)
에서	(4.888, 0.6141)


하지만 `트와이스` 의 경우에 be_r 은 크지만 be_l 이 항상 0 입니다. 이는 우리가 L-R 카운팅을 할 때, 한 어절 내의 subwords 만 확인했기 때문입니다.

```
트와이	(0.0, -0.0)
트와이스	(-0.0, 1.283)
트와이스는	(0.0, 0.0)
```

아래와 같은 문장에서 `트와이스`의 왼쪽에 등장하는 `-서`, `-도` 가 R 에 카운팅이 되었어야 합니다.

```
뮤직뱅크에서 [트와이스]의 ...
오늘도 [트와이스]는 ..
```

```python
R['트와이스']['서'] += 1
R['트와이스']['도'] += 1
```

중국어, 일본어는 띄어쓰기가 없기 때문에 모든 subwords 에 대하여 좌, 우의 글자들의 빈도수를 계산하였습니다. 그러나 한국어에서 문장 내 띄어쓰기를 모두 제거하면 안됩니다.

아래 문장에서는 `의날` 이라는 subword 는 단어 후보가 되지 않습니다.

```
오늘의 날씨는 ...
여름의 날씨가 ...
```

하지만 띄어쓰기를 제거하면 `의날`이 단어 후보로 포함됩니다. 특히 `조사 - 1글자` 의 조합이 매우 큰 Branching Entropy 값을 가질 수 있습니다.

```
오늘[의날]씨는 ...
여름[의날]씨가 ...
```

이와 같은 경우를 방지하려면 띄어쓰기가 포함된 어절에서 L 에 대해서만 단어 후보를 만들면 됩니다. [soynlp][soynlp] 의 soynlp.word.WordExtractor 에는 이를 고려하여 구현해 뒀습니다.

[soynlp]: https://github.com/lovit/soynlp/

## L-tokenizer

어절 word 가 입력되었을 때, L 들에서 cohesion 이 가장 높은 subword 를 잘라내는 토크나이저를 만들어 봅시다. 

길이가 2 이상일 때 cohesion 이 정의되기 때문에 길이가 2 이하인 단어는 그대로 return 합니다. 

subword 의 ending point e 는 길이가 2부터 'word의 길이 혹은 L의 최대 길이'의 min 까지 입니다.

In [13]:
def L_tokenize(word):

    if len(word) <= 2:
        return word

    array = []
    for e in range(2, len(word)+1):
        subword = word[:e]
        score = cohesion_score.get(subword, 0)
        # (word, score, length)
        array.append((subword, score, e))

    return sorted(array, key=lambda x:(-x[1], -x[2]))[0][0]

for word in ['청와대', '청와대는', '민정수석', '민정수석이', '트와이스', '트와이스는']:
    print('{}\t-->\t{}'.format(word, L_tokenize(word)))

청와대	-->	청와대
청와대는	-->	청와대
민정수석	-->	민정수석
민정수석이	-->	민정수석
트와이스	-->	트와이스
트와이스는	-->	트와이스


이 부분을 LTokenizer 클래스로 정리합니다. score dictionary 를 입력받아서 어절의 왼쪽에 존재하는 subwords 중 점수가 가장 큰 subword로 L + [R] 구조로 나눕니다. 

scores 는 cohesion 뿐 아니라, 임의의 방법이 모두 적용될 수 있습니다. 또한 도메인 특성에 따라 다양한 후처리 기능들도 넣을 수 있습니다.

In [14]:
class LTokenizer:
    
    def __init__(self, scores=None):
        self.scores = scores if scores else {}

    def tokenize(self, sentence, flatten=True, remove_r=False):
        words = [self._eojeol_to_lr(token) for token in sentence.split()]
        if remove_r:
            words = [token[0] for token in words]
        if (flatten) and (remove_r == False):
            words = [subword for token in words for subword in token if subword]
        return words

    def _eojeol_to_lr(self, token):
        if len(token) <= 2:
            return token

        array = []
        for e in range(2, len(token)+1):
            l, r = token[:e], token[e:]
            score = cohesion_score.get(l, 0)
            # (l, r, score, length)
            array.append((l, r, score, e))

        best = sorted(array, key=lambda x:(-x[2], -x[3]))[0]

        # return (l, r)
        return best[0], best[1]

해당 도메인에 대하여 여러분이 가지고 있는 단어 사전이 있다면 그 사전의 단어의 점수들은 모두 최대값을 지내도록 할 수도 있습니다.

In [15]:
# custom score
cohesion_score['아이오아이'] = 1.0

# create tokenizer
tokenizer = LTokenizer(scores = cohesion_score)

# test
sent = '뉴스의 기사를 이용했던 예시입니다'
tokenizer.tokenize(sent)

['뉴스', '의', '기사', '를', '이용', '했던', '예시', '입니다']

디버깅 혹은 어절 구조의 확인을 위하여 flatten 기능을 넣을 수 있습니다.

In [16]:
tokenizer.tokenize(sent, flatten=False)

[('뉴스', '의'), ('기사', '를'), ('이용', '했던'), ('예시', '입니다')]

어절의 왼쪽 부분만 필요하다면 어절의 오른쪽 부분을 제거하는 기능도 넣어주면 좋습니다.

Term frequency matrix 를 만들 때 어절의 왼쪽만 남겨도 의미가 있는 subwords 가 포함되기 때문에 문서군집화나 문서판별은 거의 잘 됩니다.

In [17]:
tokenizer.tokenize(sent, remove_r=True)

['뉴스', '기사', '이용', '예시']